This file predicts drug-target interactions using M2D2 stage 1 ML. The BindingDB database is used for training data. Instead of drugs any chemical compound with a SMILES structure can be used. The BindingDB database is primarily full of drug - protein interaction data. 

In this file, we read in drugs (or chemical compound of interest), target of interest (proteins), generate drug-protein interaction model from the BindingDB dataset. Use the model to predict drug-target protein interaction.

Input files required:
Encoding for compounds of interest (this code uses MACCS keys --> "original_MACCS.pkl")
Encoding for target of interest (this code is written for proteins encoded with PseaudoAAC --> "ecoli_4087.pkl") 




In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.metrics import PrecisionRecallDisplay
import json

In [6]:
target_data = pd.read_pickle("ecoli_4087_pseudoAAC.pkl")
# remove any duplicate sequences
# final format rows: proteins, columns: Target Sequence, PseudoAAC
# each protein has one Target Sequence amino acid sequence and 30 encodings that make up the PseudoAAC column
target_map = target_data.drop_duplicates(subset=['Target Sequence'])

In [8]:
drug_map = pd.read_pickle("drugs_MACCS.pkl")
# final format rows: compounds columns: Full name, docking abbrev (3/4 letter code), SMILES structure, MACCS encoding (67 values for each compound)

In [11]:
# combine compound and target encodings. MAACS for compounds, PseudoAAC for targets
drug_encoding = "MACCS"
target_encoding = "PseudoAAC"
drug_identifier_name = "docking abbrev"
drug_SMILES = "SMILES"
drug_df_arr=[]

for i in range(len(drug_map)):
  # Can print names to ensure you have all the compounds  required
  # print(drug_map[drug_identifier_name][i])
  new_df = target_map.copy() # copy pseudoAAC
  new_df["SMILES"] = drug_map[drug_SMILES][i]
  new_encoding = np.tile(drug_map[drug_encoding][i], (len(target_map),1))  # Copy MACCS
  new_df[drug_encoding] = new_encoding.tolist()
  new_df["drug"] = drug_map[drug_identifier_name][i]
  drug_df_arr.append(new_df)

In [12]:
# drug_df_arr is a list of dataframes for each drug. Each of the dataframes contains the protein target sequence, PseudoAAC (protein encoding), SMILES/MACCS encoding for that dataframe's drug
# note that the SMILES/MACCS repeat for each protein
# example for first two proteins of first drug of interest
drug_df_arr[1].head(2)

,Target Sequence,PseudoAAC,SMILES,MACCS,drug
0,MKRISTTITTTITITTGNGAG,"[0.03, 0.03, 0.03, 0.0, 0.0, 0.0, 0.0, 0.09, 0...",C1C(C(C(C(C1NC(=O)C(CCN)O)OC2C(C(C(C(O2)CO)O)N...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AMK
1,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,"[0.056, 0.028, 0.023, 0.027, 0.008, 0.033, 0.0...",C1C(C(C(C(C1NC(=O)C(CCN)O)OC2C(C(C(C(O2)CO)O)N...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AMK


In [13]:
# check # of compounds in dataset
len(drug_df_arr)

58

In [14]:
# specific labels in training data pkl file, 
# note that there are other encodings available for training data
training_drug_encoding = "MACCS"
training_protein_encoding = "PseudoAAC(target)"

In [15]:
# training data from the BindingDB database
# training data format: each row is a compound + target
# column 1+2: compounds in SMILES, targets in Target Sequences (amino acid sequences)
# column 3: Label (binding affinity, i.e. what we want to predict for test compounds)
# remaining columns: all possible encodings for compound + target
# encoding options:
# Target: Transformer(target), Conjoint_triad(target), PseudoAAC(target)
# Compound: Morgan, Pubchem, Transformer, ErG, Daylight, MACCS
df = pd.read_pickle("bindingdb_merged_all.pkl")

df.head(2)

,SMILES,Target Sequence,Label,Transformer(target),Conjoint_triad(target),PseudoAAC(target),Morgan,Pubchem,Transformer,ErG,Daylight,MACCS
0,Cc1ccc(CNS(=O)(=O)c2ccc(s2)S(N)(=O)=O)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.46,"([139, 270, 224, 108, 211, 81, 65, 717, 2351, ...","[0, 3, 1, 1, 0, 2, 0, 2, 3, 0, 2, 3, 2, 0, 2, ...","[0.025, 0.014, 0.019, 0.037, 0.002, 0.025, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","([757, 286, 178, 965, 22, 763, 1049, 123, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.3, 1.0, 0.3, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,COc1ccc(CNS(=O)(=O)c2ccc(s2)S(N)(=O)=O)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,0.49,"([139, 270, 224, 108, 211, 81, 65, 717, 2351, ...","[0, 3, 1, 1, 0, 2, 0, 2, 3, 0, 2, 3, 2, 0, 2, ...","[0.025, 0.014, 0.019, 0.037, 0.002, 0.025, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","([467, 286, 178, 965, 22, 763, 1049, 123, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.3, 1.0, 0.3, 0.0, 0.0, ...","[0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
# TRAINING
# note that drug and target encoding are defined above
def apply_func_train(x):
  try:
    # Concatenate drug encoding and target (protein) encoding
    combined = np.concatenate((x[training_drug_encoding],x[training_protein_encoding]))
    error_count = 0
    return combined, error_count

  except:
    # if a drug or target encoding missing note the error for deletion later
    error_count = 1
    return "error", error_count


def load_and_process(dataset):
  # reads in the training data file
  df = pd.read_pickle("bindingdb_merged_all.pkl")
  # drops rows where either 'drug_encoding' or 'training_protein_encoding' is NaN
  df.dropna(subset=[drug_encoding,training_protein_encoding],inplace=True)
  # combines drug and protein encoding or returns error
  df[['combined','error']] = df.apply(lambda x: pd.Series(apply_func_train(x)),axis=1)

  # train_nonzero=train[train['Label']!=0]
  # Filter out rows with 'error'
  if (df['error'] == 1).any():
    df = df[df['error']!=1]

  # Convert features to numpy arrays
  X=df.combined.tolist()
  X=np.array(X)
  Y=df['Label'].tolist()
  y=np.array(Y)

  # Print the binary labels
  return X,Y

# Load and process data from "bindingdb" dataset
# X : encodings for compound + target
# Y : binding affinity (Label)
X,Y = load_and_process("bindingdb")

In [18]:
mydict = {}

X_train = X
Y_train = Y
classifier = RandomForestRegressor(n_estimators=100)
classifier.fit(X_train, Y_train)

RandomForestRegressor()

In [19]:
def apply_func(x):
  try:
    combined = np.concatenate((x[drug_encoding],x[target_encoding]))
    error_count = 0
    return combined, error_count
  except:
    error_count = 1
    return "error", error_count

# for each compound/target pair
for test_df in drug_df_arr:

  # drop NaN rows
  test_df.dropna(subset=[drug_encoding,target_encoding],inplace=True)

  # combine protein and drug encodings via function above
  test_df[['combined','error']] = test_df.apply(lambda x: pd.Series(apply_func(x)),axis=1)

  if (test_df['error'] == 1).any():
    test_df = test_df[test_df['error']!=1]

  X_test=test_df.combined.tolist()
  X_test=np.array(X_test)
  y_pred = classifier.predict(X_test)
  mydict[test_df.drug.iloc[0]] = y_pred

In [21]:
# Convert dictionary to DataFrame
df = pd.DataFrame(mydict)

# Save DataFrame to CSV
df.to_csv("out.csv" )